In [ ]:
from web_scraping import Scraper
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

driver_path = "YOUR DRIVER PATH HERE"
scraper = Scraper(driver_path)

# 1. get to a page of the website and obtain all artists pages
website = "https://www.copia-di-arte.com/a/artisti/&mpos="
artists_urls = [] 
pages = []
images_links_correct = []
# looping over all alphabetical pages
for i in range(1000, 1026):  #1026
    # getting all hrefs referred to artist names
    artists_urls.extend(scraper.find_hrefs(website+str(i), scroll=False, subset_links='.html&mpos='))
# for each artist page I get all the sub-pages (ex: raffaello page 1, raffaello page 2) 
for artist in artists_urls:
    # getting all the pages
    pages.extend(scraper.find_hrefs(artist, scroll=False, subset_links='.html?pgn_page='))
# saving pages

for page in pages:
    # per ogni pagina ottengo un subset, cioè le pagine con author name. le altre sono thrash
    subset = page.split(".html")[0].split('/')[-1] + "/" #!!!
     # prendo gli href dalla pagina che contengono il nome dell'artista: sono le immagini
    images_links = scraper.find_hrefs(page, subset_links=subset)
    # tengo solo quelle che finiscono con html: le altre sono thrash
    images_links_correct.extend([image for image in images_links if image[-4:]=='html']) 

# first checkpoint
with open("images_links_correct.txt","w") as f:
    for line in images_links_correct: 
        f.write(line + '\n')
        
        
f = open("images_links_correct.txt", 'r')
file = f.read()
images_links_correct = file.splitlines()
images_links_correct = list(set(images_links_correct))
print(len(images_links_correct))


links = [] 
artists = []    
art_movement = []
title = []
for i in range(len(images_links_correct)):  
    img_link = images_links_correct[i]
    scraper.get_url(img_link)
    try:
        elements = scraper.find_classes(url=img_link, tag="div", class_name="val")
        #element 0 is artist name
        if len(elements) != 0:
            links.append(img_link)
            artists.append(elements[0].replace("\n","").replace("\t",""))
            #element 1 is artistic movement
            art_movement.append(elements[1].replace("\n","").replace("\t",""))
            #element 2 is title
            title.append(elements[2].replace("\n","").replace("\t",""))
    except:
        pass

result = pd.DataFrame({"image_link":links, "artists":artists, "art_movement": art_movement, "title":title})
result.to_csv("links_final.csv")